Installing Required Libraries

In [1]:
!pip install flask pyngrok requests openai google-auth nest-asyncio

Importing Libraries

In [2]:
from flask import Flask, render_template_string, request
from pyngrok import ngrok
import threading
import nest_asyncio
import requests
from openai import OpenAI
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os
from datetime import datetime, timezone , timedelta
import json
import time
import logging
import traceback

API Section

In [3]:
os.environ['GOOGLE_MAPS_API_KEY'] = "Enter your API Key here"
os.environ['OPENAI_API_KEY'] = "Enter your API Key here"

Google OAuth Process

In [4]:
from google.colab import files
uploaded = files.upload()

Saving precemap-ee1cbbc06eef.json to precemap-ee1cbbc06eef (10).json


In [5]:
SERVICE_ACCOUNT_FILE = "/content/precemap-ee1cbbc06eef.json"
SCOPES = ["https://www.googleapis.com/auth/generative-language"]

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=SCOPES
)

credentials.refresh(Request())
GEMINI_ACCESS_TOKEN = credentials.token
print("Gemini OAuth access token acquired.")

Gemini OAuth access token acquired.


OPENAI Client

In [6]:
# Initialize the OpenAI client once (do this globally)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [19]:
app = Flask(__name__)
html_template = '''
<!DOCTYPE html>
<html>
<head>
    <title>Route Analysis</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            padding: 0 10px;
            max-width: 600px;
            margin-left: auto;
            margin-right: auto;
        }
        input, button {
            padding: 10px;
            margin: 10px 0;
            width: 100%;
            box-sizing: border-box;
            font-size: 1rem;
        }
        button {
            background-color: #0b75c9;
            color: white;
            border: none;
            cursor: pointer;
            max-width: 100%;
        }
        pre {
            background: #f4f4f4;
            padding: 15px;
            white-space: pre-wrap;
            word-wrap: break-word;
            max-height: 400px;
            overflow: auto;
            border-radius: 5px;
            box-shadow: 0 1px 4px rgba(0,0,0,0.1);
        }
        h2, h3 {
            text-align: center;
        }
        form {
            display: flex;
            flex-direction: column;
        }
        /* Responsive adjustments */
        @media (max-width: 480px) {
            body {
                margin: 10px;
                padding: 0 5px;
                max-width: 100%;
            }
            input, button {
                font-size: 1rem;
            }
        }
    </style>
</head>
<body>
    <h2>Enter Route Details</h2>
    <form method="POST" action="/">
        Origin:<br>
        <input type="text" name="origin" required><br>
        Destination:<br>
        <input type="text" name="destination" required><br>
        Departure Time (optional):<br>
        <input type="datetime-local" name="departure_time"><br>
        <button type="submit">Get Best Route</button>
    </form>

    {% if result %}
    <h3>Analysis Result:</h3>
    <pre>{{ result }}</pre>
    {% endif %}
</body>
</html>
'''

Geocoding Addresses

In [8]:
def geocode_address(address):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": address,
        "key": os.getenv("GOOGLE_MAPS_API_KEY")
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        return location["lat"], location["lng"]
    else:
        raise Exception(f"Geocode error: {data.get('error_message', data['status'])}")

def get_routes(origin_addr, destination_addr, departure_time=None):
    url = "https://routes.googleapis.com/directions/v2:computeRoutes"
    api_key = os.getenv("GOOGLE_MAPS_API_KEY")

    origin_lat, origin_lng = geocode_address(origin_addr)
    dest_lat, dest_lng = geocode_address(destination_addr)

    field_mask = "routes.legs.distanceMeters,routes.legs.duration,routes.legs.steps.distanceMeters,routes.legs.steps.duration,routes.legs.steps.maneuver,routes.legs.steps.startLocation,routes.legs.steps.endLocation"

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "*"
    }

    if departure_time is None:
        departure_time = (datetime.now(timezone.utc) + timedelta(minutes=5)).isoformat()

    body = {
        "origin": {
        "location": {
            "latLng": {
                "latitude": origin_lat,
                "longitude": origin_lng
            }
        }
    },
    "destination": {
        "location": {
            "latLng": {
                "latitude": dest_lat,
                "longitude": dest_lng
            }
        }
    },
    "travelMode": "DRIVE",
    "computeAlternativeRoutes": True,
    "departureTime": departure_time,
    "routingPreference": "TRAFFIC_AWARE"
    }

    response = requests.post(url, headers=headers, json=body)
    response_json = response.json()
    print(json.dumps(response_json, indent=2))
    return response_json

Route Summary

In [9]:
def summarize_routes(route_response, max_routes=3):
    summaries = []
    for idx, route in enumerate(route_response.get("routes", [])[:max_routes]):
        dist_m = route.get("distanceMeters", 0)
        dur_val = route.get("duration", "0s")
        if isinstance(dur_val, str):
            dur_s = int(dur_val.rstrip("s"))
        else:
            dur_s = int(dur_val)
        dist_km = dist_m / 1000
        dur_min = dur_s / 60
        summary = f"Route {idx+1}: Distance approximately {dist_km:.1f} km, estimated time {dur_min:.1f} minutes."
        summaries.append(summary)
    summary_text = "\n".join(summaries)
    return summary_text

Route Details Extraction

In [10]:
def extract_route_details(route_response, max_routes=3):
    route_details_list = []
    for idx, route in enumerate(route_response.get("routes", [])[:max_routes]):
        route_steps = []
        for leg in route.get("legs", []):
            for step in leg.get("steps", []):
                step_detail = {
                    "roadName": step.get("roadName"),
                    "maneuver": step.get("maneuver"),
                    "lengthMeters": step.get("lengthMeters"),
                    "duration": step.get("duration"),
                    "exitNumber": step.get("exitNumber"),
                }
                route_steps.append(step_detail)
        route_details_list.append({"routeIndex": idx + 1, "steps": route_steps})
    return route_details_list

Detailed Route Summary

In [11]:
def summarize_route_details(route_details, max_steps_per_route=10):
    summaries = []
    for route in route_details:
        idx = route["routeIndex"]
        steps = route["steps"][:max_steps_per_route]  # cap steps to keep prompt manageable

        parts = []
        for step in steps:
            road = step.get("roadName", "Road")
            length_km = step.get("lengthMeters", 0) / 1000
            maneuver = step.get("maneuver", "")
            exit_num = step.get("exitNumber")

            part = f"Drive on {road}"
            if length_km > 0:
                part += f" for {length_km:.1f} km"
            if exit_num:
                part += f", take exit {exit_num}"
            if maneuver:
                part += f", then {maneuver}"
            parts.append(part)

        route_summary = f"Route {idx} details:\n" + "\n".join(parts)
        summaries.append(route_summary)

    return "\n\n".join(summaries)

Navigation Instructions Extraction

In [12]:
def extract_navigation_instructions(route_response, max_routes=3):
    instructions = []
    for idx, route in enumerate(route_response.get("routes", [])[:max_routes]):
        steps_texts = []
        for leg in route.get("legs", []):
            for step in leg.get("steps", []):
                nav_instr = step.get("navigationInstruction", {})
                instr = nav_instr.get("instructions", "")  # Extract your target 'instructions' string
                if instr:
                    steps_texts.append(instr)
        instructions.append(f"Route {idx+1} Navigation Instructions:\n" + "\n".join(steps_texts))
    return "\n\n".join(instructions)

GPT and Gemini Prompt Creation

In [13]:
def create_prompt_with_navigation(route_data, mode="car", priorities="fastest time", departure_time=None, max_routes=3, max_steps=10):
    summary = summarize_routes(route_data, max_routes=max_routes)
    navigation_instructions = extract_navigation_instructions(route_data, max_routes=max_routes)

    if departure_time:
        dt_str = datetime.datetime.fromtimestamp(departure_time).strftime("%Y-%m-%d %I:%M %p")
    else:
        dt_str = "current time"

    prompt = (
        f"I am traveling by {mode} departing at {dt_str}.\n"
        f"My priority is {priorities}.\n"
        f"Here are the routes provided by Google Maps with distances and times:\n{summary}\n\n"
        f"Step-by-step driving instructions for each route:\n{navigation_instructions}\n\n"
        "Based on this, suggest any alternative or optimized routes, specifying which highways and local roads I should take, "
        "including any exits or maneuvers that could improve the route for faster and more accurate travel."
    )
    return prompt

AI Route Analysis with GPT

In [14]:
def analyze_route_gpt(route_data):
    prompt = create_prompt_with_navigation(route_data)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=200
    )
    return response.choices[0].message.content

AI Route Analysis with Gemini

In [15]:
def analyze_route_gemini(route_data):
    prompt = create_prompt_with_navigation(route_data)
    gemini_endpoint = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
    headers = {
        "Authorization": f"Bearer {GEMINI_ACCESS_TOKEN}",
        "Content-Type": "application/json",
    }
    payload = {"contents": [{"parts": [{"text": prompt}]}]}
    response = requests.post(gemini_endpoint, headers=headers, json=payload)
    response_json = response.json()
    try:
        return response_json['candidates'][0]['content']['parts'][0]['text']
    except (KeyError, IndexError):
        return "No response from Gemini or unexpected response format."

Synthesis of AI Suggestions

In [16]:
def synthesize_routes(gpt_res, gemini_res):
    return f"GPT suggests:\n{gpt_res}\n\nGemini suggests:\n{gemini_res}"

Flask Web Server and UI

In [17]:
@app.route("/", methods=["GET", "POST"])
def home():
    result = None
    error_message = None
    try:
        if request.method == "POST":
            origin = request.form.get("origin")
            destination = request.form.get("destination")
            departure_time = request.form.get("departure_time")

            # Validate required fields
            if not origin or not destination:
                error_message = "Origin and destination are required."
                return render_template_string(html_template, result=error_message)

            if departure_time:
                try:
                    dt = datetime.datetime.fromisoformat(departure_time)
                    departure_ts = int(dt.timestamp())
                except Exception as e:
                    error_message = f"Invalid departure time format: {e}"
                    return render_template_string(html_template, result=error_message)
            else:
                departure_ts = None

            route_data = get_routes(origin, destination, departure_ts)

            # Check if routes exist in response
            if not route_data.get("routes"):
                error_message = "No routes found for the given locations."
                return render_template_string(html_template, result=error_message)

            gpt_result = analyze_route_gpt(route_data)
            gemini_result = analyze_route_gemini(route_data)
            result = synthesize_routes(gpt_result, gemini_result)

    except Exception as exc:
        logging.error(f"Exception in home route: {exc}")
        logging.error(traceback.format_exc())
        error_message = f"Internal server error: {exc}"

    return render_template_string(html_template, result=result or error_message)

In [18]:
import nest_asyncio
nest_asyncio.apply()

public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

def run():
    app.run(host="0.0.0.0", port=5000)

import threading
thread = threading.Thread(target=run)
thread.start()

Public URL: NgrokTunnel: "https://9bf43a31fdb6.ngrok-free.app" -> "http://localhost:5000"
